# Restaurant

## Import

In [48]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import accuracy_score
from neo4j.v1 import GraphDatabase, basic_auth

In [49]:
refs_utts = pd.read_pickle('utts_refs.pkl')
props = pd.read_pickle('restaurants_props.pkl')
len(refs_utts), len(props)

(414, 52256)

In [50]:
refs_utts[:5]

,text,bot,o,ind,mask,gid,target
0,"[i, want, a, moderately, priced, restaurant, i...",api_call R_cuisine west moderate,trn,2,True,2,prezzo
2,"[cheap, restaurant, in, the, north, part, of, ...",api_call R_cuisine north cheap,trn,2,True,11,da_vinci_pizzeria
3,"[cheap, restaurant, in, the, south, part, of, ...",api_call R_cuisine south cheap,trn,2,True,12,the_lucky_star
4,"[cheap, restaurant, serving, indian, food]",api_call indian R_location cheap,trn,2,True,15,the_gandhi
5,"[thai, food]",api_call thai R_location R_price,trn,2,True,22,bangkok_city


In [89]:
props[:5]

,rname,attr_key,attr_value
3,saint_johns_chop_house,R_cuisine,british
4,saint_johns_chop_house,R_location,west
7,saint_johns_chop_house,R_price,moderate
10,prezzo,R_cuisine,italian
11,prezzo,R_location,west


## Process Text

In [230]:
stemmer = nltk.stem.snowball.EnglishStemmer()

def stem(sentence):
    return [stemmer.stem(w) for w in sentence]

In [231]:
test = pd.DataFrame()
test['text'] = [stem(s) for s in refs_utts.text]
test['frame'] = [tuple(stem(f.split()[1:])) for f in refs_utts.bot]
len(test)

414

In [232]:
# Remove poorly formatted frames
test = test[test.frame.map(len) == 3]
len(test)

405

In [233]:
test[:5]

,text,frame
0,"[i, want, a, moder, price, restaur, in, the, w...","(r_cuisin, west, moder)"
1,"[cheap, restaur, in, the, north, part, of, town]","(r_cuisin, north, cheap)"
2,"[cheap, restaur, in, the, south, part, of, town]","(r_cuisin, south, cheap)"
3,"[cheap, restaur, serv, indian, food]","(indian, r_locat, cheap)"
4,"[thai, food]","(thai, r_locat, r_price)"


In [234]:
knowledge = pd.DataFrame()
knowledge['restaurant'] = props.rname.copy()
knowledge['key'] = [stemmer.stem(s) for s in props.attr_key]
knowledge['value'] = [stemmer.stem(s) for s in props.attr_value]

In [235]:
knowledge[:5]

,restaurant,key,value
3,saint_johns_chop_house,r_cuisin,british
4,saint_johns_chop_house,r_locat,west
7,saint_johns_chop_house,r_price,moder
10,prezzo,r_cuisin,italian
11,prezzo,r_locat,west


In [236]:
# A dictionary of keys to the list of values they can take
# In this instance, keys form mutually exclusive lists of values
types = knowledge[['key', 'value']] \
    .groupby('key') \
    .aggregate(lambda x: tuple(set(x))) \
    .reset_index() \
    .set_index('key') \
    .value \
    .to_dict()

In [237]:
types['r_cuisin'][:5]

('bistro', 'african', 'turkish', 'italian', 'korean')

In [238]:
types['r_locat']

('north', 'east', 'south', 'west', 'centr')

In [239]:
types['r_price']

('cheap', 'expens', 'moder')

## Create Knowledge Graph

In [189]:
# Create a neo4j session
driver = GraphDatabase.driver('bolt://localhost:7687', auth=basic_auth('neo4j', 'neo4j'))

In [248]:
# WARNING: This will clear the database when run!
def reset_db():
    session = driver.session()
    session.run('MATCH (n) DETACH DELETE n')

In [249]:
reset_db()

In [250]:
session = driver.session()

for i,row in knowledge.iterrows():
    subject, relation, obj = row.restaurant, row.key, row.value
    session.run('''
        MERGE (s:SUBJECT {name: $subject}) 
        MERGE (o:OBJECT  {name: $obj}) 
        MERGE (s)-[r:RELATION {name: $relation}]->(o)
    ''', { 
        'subject': subject,
        'relation': relation,
        'obj': obj
    })

## Test
#### Baseline
The baseline accuracy is calculated by the assumption of not knowing any frame values for any of the sentences.

In [240]:
dont_know = tuple(types.keys())
dont_know

('r_cuisin', 'r_locat', 'r_price')

In [241]:
base_predicted = list(dont_know) * len(test)
base_actual = [w for frame in test.frame for w in frame]

In [245]:
accuracy_score(base_actual, base_predicted)

0.45267489711934156

#### Accuracy

In [ ]:
# TODO: Cache properties from DB
# TODO: Hotword listener
# TODO: Issue DB queries